<a href="https://colab.research.google.com/github/Jit562/GenAI-Q---A/blob/main/Llama_2_%2B_Langchain_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the libraries

In [ ]:
!pip install -q langchain transformers accelerate bitsandbytes

Load the libraries

In [ ]:
!pip install matplotlib-venn

In [ ]:
!pip install --upgrade langchain transformers

In [ ]:
!pip install --upgrade langchain

In [ ]:
!pip install langchain[huggingface]

In [ ]:
!pip install langchain_community

In [ ]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain


from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap

Download the Model - We are using NousResearch's Llama2 which is the same as Meta AI's Llama 2, the only difference being "**Not requiring authentication to download**"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")

In [ ]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Define Transformers Pipeline which will be fed into Langchain

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


Define the Prompt format for Llama 2 - This might change if you have a different model

In [ ]:

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an advanced Life guru and mental health expert that excels at giving advice.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!"""


All the helper fucntions to generate prompt, prompt template, clean up output text

In [ ]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

Defining Langchain LLM

In [ ]:
!pip install transformers

In [ ]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})



In [ ]:
system_prompt = "You are an advanced Life guru and mental health expert that excels at giving advice. please do not use any emoji "
instruction = "Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

[INST]<>
You are an advanced Life guru and mental health expert that excels at giving advice. please do not use any emoji 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:

 {text}[/INST]


In [ ]:
prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)



In [ ]:
text = "i am suffreing from feaver can you tell me some precautaion can you suggest some medicine"

In [ ]:
response = llm_chain.run(text)
print(response)

[INST]<>
You are an advanced Life guru and mental health expert that excels at giving advice. please do not use any emoji 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:

 i am suffreing from feaver can you tell me some precautaion can you suggest some medicine[/INST]  Certainly, I'm here to help! 😊

Firstly, I must commend you on taking the initiative to seek medical advice. It's essential to prioritize your health, especially when you're experiencing symptoms like fever. 💖

To address your concern, here are some precautions and medication suggestions for managing fever:

1. Precautions:
a. Rest and relaxation: Fever can be exhausting, so make sure to get plenty of rest and relaxation. Try to avoid strenuous activities and take breaks when needed. 😴
b. Stay hydrated: Drink plenty of fluids, such as water, clear broths, or electrolyte-rich drinks like coconut water or sports drinks. This will help replace lost fluids and 